### Decision trees

 决策树是一种基本的分类与回归方法。它可以认为是if-then 规则的集合，也可以是认为是定义在特征空间与类空间上的条件概率分布。
 
 ### 它包括三个步骤
 
 特征选择
 
 决策树的生成
 
 决策树的修剪

### 决策树模型

决策树由结点和有向边组成，不能再往下分的叫叶节点，能往下分的叫做内部结点。


In [1]:
import numpy as np
import pandas as pd #基于numpy的数据分析包，利用pandas进行运算

In [2]:
#加载数据集
dataSet = [['y','y','y'],
              ['y','y',"y"],
              ['y','n',"n"],
              ['n','y',"n"],
              ['n','y',"n"]]
labels = ['no surfacing','flippers','lables']
dataSet = np.array(dataSet) #转换成向量的形式
labels = np.array(labels)   #只有转换成向量的形式才能计算
dataSet

array([['y', 'y', 'y'],
       ['y', 'y', 'y'],
       ['y', 'n', 'n'],
       ['n', 'y', 'n'],
       ['n', 'y', 'n']], dtype='<U1')

In [3]:
#转换成列表的形式
data = pd.DataFrame(data = dataSet,columns = labels)
data

,no surfacing,flippers,lables
0,y,y,y
1,y,y,y
2,y,n,n
3,n,y,n
4,n,y,n


In [56]:
def xiangnongzhi(data):
    labels_data = data.iloc[:,-1]
    labels_size = labels_data.size
    zidian = {}
    list = []
    for i in labels_data:
        if i not in zidian:
            zidian[i] = 1
        else:
            zidian[i] += 1
    for key,value in zidian.items():
        list.append(value)
        a = np.array(list) / labels_size
        b = np.sum(-a * np.log2(a))
    return b

In [57]:
xiangnongzhi(data)

0.9709505944546686

In [60]:
def zuiyou(data):
    a = data.index.size
    fetures = data.columns[:-1]
    c_dic = {}
    esum = 0
    for i in fetures:
        data1 = data[i]
        lei = pd.unique(data1)
        for j in lei:
            data_son = data[data1 == j]
            fenzi = data_son.index.size
            ent = xiangnongzhi(data_son) * fenzi / a
            esum += ent
        c_dic[i] = esum
    zuiyou = sorted(c_dic.items(),key = lambda z:z[1] , reverse=False)[0][0]        
    return zuiyou
    
    

In [62]:
zuiyou = zuiyou(data)
zuiyou

'no surfacing'

#看标签中一共有几个样本，作为计算总信息熵的分母
H_y_fenmu = data.index.size 
H_y_fenmu

H_y_yes = (data.iloc[:,-1] == 'y').sum()
H_y_yes

H_y_no = (data.iloc[:,-1] == 'n').sum()
H_y_no

H_y_pi = np.array([H_y_yes/H_y_fenmu,H_y_no/H_y_fenmu])
H_y_pi

lables_data = data.iloc[:,-1] 
lables_data

In [9]:
#第一步 计算香农值
def calcShannonEnt_labels(data):
    labels_data = data.iloc[:,-1]  #取出最后一列
    labels_size = labels_data.size  #返回的是5 最后的总的标签数
    #统计各个种类，以及不同种类数
    dict_label = {}#定义一个字典 往里面放标签的种类
    
    #定义一个空列表，放入值
    H_y = []
    
    #构建一个循环
    for i in labels_data:
        if i not in dict_label: #if i 不在那个字典里，就把i放进去
            dict_label[i] = 1
        else:                   #if i 在这个字典里，自加1，起到一个计数的作用
            dict_label[i] += 1
            
    #>>>print(dict_label)
    #>>>{'y': 2, 'n': 3}
            
    #再次构建一个循环，提取并运用里面的数字
    #字典(Dictionary) items() 函数以列表返回可遍历的(键, 值) 元组数组
    for key,value in dict_label.items():
        H_y.append(value)
    H_y = np.array(H_y) / labels_size 
    H_y = np.sum(-H_y * np.log2(H_y)) #得出的结果是总的香农值 也就是y的概率和n的概率放在一起的香农值
    return H_y

In [10]:
calcShannonEnt_labels(data)

0.9709505944546686

In [105]:
#求得是一个最优的变量
def calcShannonEnt_feture(data):
    data_size = data.index.size #求总的样本数，此案例为5
    fetures = data.columns[:-1] #获取列标签，除了最后一列
    
    #>>>print(fetures)
    #>>>Index(['no surfacing', 'flippers'], dtype='object')
    
    #columns[:-1]中间没有逗号
    best_feture_dic = {} 
    for feture in fetures: #循环列中的每一列
        feture_son = data[feture] #获取数据中的特征的列,每一列的特征值
        unique_feture = pd.unique(feture_son) #取出列中唯一的特征值,除了最后一列
        
        #>>>print(unique_feture)
        #>>>['y' 'n']
        #   ['y' 'n']
        
        #>>>print(feture_son)
        #>>>获取的是每一个特征的特征向量
        
        #计算的是最优的
        Ent_sum = 0 
        for u_feture in unique_feture:
            
            #分别按照x1和x2进行y和n的划分
            data_son = data[feture_son == u_feture]
            
            #print(data_son)
                  #no surfacing flippers lables
                #0            y        y      y
                #1            y        y      y
                #2            y        n      n
                 # no surfacing flippers lables
                #3            n        y      n
                #4            n        y      n
                # no surfacing flippers lables
                #0            y        y      y
                #1            y        y      y
                #3            n        y      n
                #4            n        y      n
                  #no surfacing flippers lables
                #2            y        n      n
            fenzi = data_son.index.size  
            #print(fenzi)
            #3 2 4 1
            
            Ent = calcShannonEnt_labels(data_son) * fenzi / data_size 
            Ent_sum += Ent   
        
        best_feture_dic[feture] = Ent_sum 
        #print(best_feture_dic)
    best_feture = sorted(best_feture_dic.items(),key=lambda z:z[1],reverse=False)[0][0]  #按照第二个元素排列 升序
    
    return best_feture

In [106]:
calcShannonEnt_feture(data)

'no surfacing'

In [107]:
best_feture = calcShannonEnt_feture(data)
best_feture

'no surfacing'

In [108]:
def split_Tree(data,best_feture):
    best_feture_son = pd.unique(data[best_feture])
    #print(best_feture_son)
    split_result = []
    sum_data = 0
    for feture_son in best_feture_son:
        #循环最优的特征列的y和n
        split_data = data[data[best_feture]==feture_son] 
        split_data_label = split_data.iloc[:,-1] #取出最优列的最后一列，也就是标签的那列
        
        #按照特征列划分之后标签中的类别
        unique_size = pd.unique(split_data_label).size
       
        if unique_size == 1: #如果划分后的标签类别只有一个
            
            #>>>print([best_feture,feture_son])
            #>>>['no surfacing', 'n']
            
            split_result.append([best_feture,feture_son]) #加到那个空列表中
            sum_data += split_data.index.size #计数有几个
            
            #>>>print(sum_data)
            #>>>2
            
        else: #如果不止一个类别
            
            drop_data = split_data.drop(columns = best_feture) #删掉最优的那一列，剩下还可以继续划分的特征
            #print(drop_data)
            split_result.append([best_feture,feture_son,drop_data])
    return split_result,sum_data

In [109]:
split_Tree(data,best_feture)

([['no surfacing', 'y',   flippers lables
   0        y      y
   1        y      y
   2        n      n], ['no surfacing', 'n']], 2)

In [110]:
def DT(data):
    Split_result = []
    all_data = data.index.size
    all_data_columns = data.columns[:-1].size
    sum_data = 0
    sum_labels = 0
    while 1:
        # 所有的样本已经划分完毕
        if sum_data == all_data:
            print('样本划分完毕')
            return Split_result
        if sum_labels == all_data_columns:
            if sum_data == all_data:
                print('标签使用完毕,且样本完全划分完毕')
                return Split_result
            else:
                print('标签使用完毕,但是样本未完全划分')
                return Split_result
            
        best_feture = calcShannonEnt_feture(data)
        sum_labels +=1
        split_result,sum_data_son = split_Tree(data,best_feture)
        sum_data += sum_data_son
    
        for i in split_result:
            if len(i) == 2:
                Split_result.append(i)
            else:
                data = i[-1]
              

In [111]:
DT(data)

样本划分完毕


[['no surfacing', 'n'], ['flippers', 'y'], ['flippers', 'n']]